In [1]:
#google driveとの連携
from google.colab import drive
drive.mount('/content/drive')

#カレントディレクトリの設定
dir_path = '/content/drive/My Drive/Colab Notebooks/repo12/'
import os
os.chdir(dir_path)

#モジュール追加用pathの設定
import sys
sys.path.append('..')

#モジュールのインポート
import numpy as np

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install transformers
!apt install git make curl xz-utils file
!apt install mecab libmecab-dev mecab-ipadic mecab-ipadic-utf8
!pip install mecab-python3==0.996.5

     |████████████████████████████████| 778kB 2.7MB/s 
     |████████████████████████████████| 890kB 13.0MB/s 
     |████████████████████████████████| 3.0MB 17.5MB/s 
     |████████████████████████████████| 1.1MB 28.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=c8356eabed3127e2f0f2b28680b6075ab2710cc9440c008c951bc4499ba52126
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Reading package lists... Done
Building dependency tree       
Reading state information... Done
make is already the newest version (4.1-9.1ubuntu1).
make set to manually installed.
xz-utils is already the newest version (5.2.2-1.3).
xz-utils set to manually installed.
curl is already the newest version (7.58.0-2ubuntu3.9).
git is already the newest version (1:2.17.1-1ubuntu0.7).
The following package was automatically installed and is no longer required:
  libnvidia-common-4

In [3]:
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM
import transformers

pretrained_model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'

# 事前学習済みモデルのトークナイザを使用
tokenizer = BertJapaneseTokenizer.from_pretrained(pretrained_model_name)

In [4]:
import pickle
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence, pad_sequence, pack_padded_sequence

In [5]:
!wget http://www.rondhuit.com/download/ldcc-20140209.tar.gz
!tar xfz ldcc-20140209.tar.gz

--2020-08-23 11:57:07--  http://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.rondhuit.com/download/ldcc-20140209.tar.gz [following]
--2020-08-23 11:57:08--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz.3’

ldcc-20140209.tar.g 100%[===================>]   8.44M  7.47MB/s    in 1.1s    

2020-08-23 11:57:09 (7.47 MB/s) - ‘ldcc-20140209.tar.gz.3’ saved [8855190/8855190]



In [6]:
category2id = {"kaden-channel":0, "peachy":1, "sports-watch":2, "dokujo-tsushin":3, "livedoor-homme":4, "it-life-hack":5, "movie-enter":6}

# Livedoorニュースコーパスからテキスト分類用のデータを作成．
import glob
import random

# 1行は [文][TAB][ラベル]からなる
write_lines = []
for d in category2id.keys():
  for file in glob.glob("text/" + d + "/*.txt"):
    with open(file) as f:
      lines = f.readlines()
      # 最初の2行はURLと日付なので捨てる
      for line in lines[3:]:
        line = line.strip()
        if len(line) > 20 and len(line) < 256 and "http" not in line:
          write_lines.append(line + "\t" + str(category2id[d]) + "\n")

random.shuffle(write_lines)
# Train, Dev, Testの3つに分ける.
dev = write_lines[0:2000]
test =write_lines[2000:4000]
train = write_lines[4000:]
# ファイルに保存
w = open("./data/train.tsv", "w")
w.writelines(train)
w.close()
w = open("./data/dev.tsv", "w")
w.writelines(dev)
w.close()
w = open("./data/test.tsv", "w")
w.writelines(test)
w.close()
print("data size ", len(train), len(dev), len(test))

data size  67653 2000 2000


In [38]:
emb_dim = 300
hidden_dim = 50
target_size = 4
batch_size = 32

def get_data(fname):
    label_list = [] # ラベル(カテゴリーのid)を格納するリスト
    title_list = [] # タイトルに含まれる単語のID系列のtensorを格納するリスト

    category2id = {"kaden-channel":0, "peachy":1, "sports-watch":2, "dokujo-tsushin":3, "livedoor-homme":4, "it-life-hack":5, "movie-enter":6}

    with open(fname) as f:
        for line in f:
            if not line:
                continue
            line = line.strip()
            
            label = line.split('\t')[1]#todo lineからカテゴリーを取得
            if(len(label) != 1): continue

            title = line.split('\t')[0]#todo lineからタイトルを取得
            title_list.append(title)

            label = int(label)
            
            label_list.append(label) #todo labelをlabel_listにappend

    labels = torch.tensor(label_list) #todo label_listをtensorに変換
    return title_list, labels


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# データの準備
train_path = './data/train.tsv'#todo 学習用ファイルパス
# !head ./data/train.tsv
train_x, train_y = get_data(train_path)
valid_path = './data/dev.tsv' #todo 評価用ファイルパス
valid_x, valid_y = get_data(valid_path)

class RnnDataset(torch.utils.data.Dataset):
    def __init__(self, data, label):
        # self.length = [len(data[i]) for i in range(len(data))] #todo dataに含まれる各ID系列の長さのリスト
        self.label = label
        self.for_bert_inputs = tokenizer(data, padding=True, return_tensors="pt")
    def __len__(self):
        return len(self.for_bert_inputs["input_ids"])
    def __getitem__(self, idx):
      return self.for_bert_inputs["input_ids"][idx], self.for_bert_inputs["token_type_ids"][idx], self.for_bert_inputs["attention_mask"][idx], self.label[idx]

dataset = RnnDataset(train_x, train_y)
validset = RnnDataset(valid_x, valid_y)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(validset, batch_size=batch_size, shuffle=True)
model = transformers.BertForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=7)
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.05)
loss_fn = nn.CrossEntropyLoss()


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [39]:
def train(model, train_loader):
    model.train()
    total_loss = 0
    correct_num = 0
    len_loader = len(train_loader) * batch_size
    print("len_loader ---- ", len_loader)
    for input_ids, token_type_ids, attention_mask, label in train_loader:
        # GPUへ
        input_ids = input_ids.to(device)
        token_type_ids = token_type_ids.to(device)
        attention_mask = attention_mask.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        out = model.forward(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask) #todo modelで予測
        pred = out[0]
        # print(" pred.shape ---- ", pred.shape) #(16, 7)
        # print("label.shape ---- ", label.shape) #(16)
        loss = loss_fn(pred, label) #todo lossを計算
        loss.backward()
        optimizer.step()
        
        total_loss += loss #todo lossをtotal_lossに加算
        correct_num += (torch.argmax(pred, axis=1) == label).sum().item() #todo 予測の正解数をカウント
    print("total_loss ---- ", total_loss)
    print("len(train_loader) ---- ", len(train_loader))
    print("correct_num ---- ", correct_num)
    print("len_loader", len_loader)
    batch_loss = total_loss/len(train_loader) #バッチごとの平均ロス
    acc = correct_num/len_loader #todo 正解率
    return batch_loss, acc

def evaluation(model, valid_loader):
    model.eval()
    len_loader = len(valid_loader) * batch_size
    total_loss = 0
    correct_num = 0

    for input_ids, token_type_ids, attention_mask, label in valid_loader:
      input_ids = input_ids.to(device)
      token_type_ids = token_type_ids.to(device)
      attention_mask = attention_mask.to(device)
      label = label.to(device)
      with torch.no_grad():
        out = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)#todo 予測
        pred = out[0]
        loss = loss_fn(pred, label)#todo 損失を計算
        total_loss += loss.item() #lossをtotal_lossに加算
        correct_num += (torch.argmax(pred, axis=1)==label).sum().item() #予測の正解数をカウント
      
    batch_loss = total_loss/len(valid_loader)
    acc = correct_num/len_loader#todo 正解率を計算
    return batch_loss, acc

for epoch in range(30):
    train_loss, train_acc = train(model, train_loader)
    valid_loss, valid_acc = evaluation(model, valid_loader)

    print(f"epoch: {epoch}")
    print(f"<train> Loss: {train_loss}\tAccuracy: {train_acc}")
    print(f"<valid> Loss: {valid_loss}\tAccuracy: {valid_acc}")

len_loader ----  67648


KeyboardInterrupt: ignored